In [47]:
import sys
sys.path.insert(0, '..')
import utils.game_utils as gu

import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


def get_previous_record(year, week, team, all_games_df):
    records = all_games_df[(all_games_df['year'] == year)
                           & (all_games_df['week'] < week) &
                           (all_games_df['team'] == team)]
    if len(records) < 1:
        return None

    return records[-1:]


def get_all_previous_records(year, week, team, all_games_df):
    records = all_games_df[(all_games_df['year'] == year)
                           & (all_games_df['week'] < week) &
                           (all_games_df['team'] == team)]
    if len(records) < 1:
        return None

    return records

def trim_df(df):
    return df[[
        'year', 'week', 'team_win', 'team', 'team_wins_before',
        'team_losses_before', 'team_ties_before', 'team_wins_after',
        'team_losses_after', 'team_ties_after', 'team_record_total_before',
        'team_record_total_after', 'team_record_normal_before',
        'team_record_normal_after', 'team_schedule_strength_before',
        'team_schedule_strength_after'
    ]]

all_games_df = pd.read_csv('../../data/games/all_games_with_data.csv')


In [52]:
def get_df_with_aggregates(index, all_games_df, side='team'):
    SIDE = side # can be opponent
    OTHER_SIDE = 'opponent' if SIDE == 'team' else 'team'
    game_df = all_games_df.iloc[index]
    year = game_df['year']
    week = game_df['week']
    team = game_df[SIDE]
    opp_team = game_df[OTHER_SIDE]

    previous_df = get_previous_record(year, week, team, all_games_df)
    opp_previous_df = get_previous_record(year, week, opp_team, all_games_df)

    wins_before = 0
    losses_before = 0
    ties_before = 0

    wins_after = 0
    losses_after = 0
    ties_after = 0

    # todo: still need to handle these (before and after?)
    record_total_before = 0
    record_total_after = 0
    record_normal_before = 0
    record_normal_after = 0
    schedule_strength_before = 0
    schedule_strength_after = 0
    # accumulated normalized records of all opponents
    opp_record_normal_before = 0
    opp_record_normal_after = 0



    ## INFO FROM THIS GAME
    win = bool(game_df[f'{SIDE}_win'])
    tie = game_df['team_score'] == game_df['opponent_score'] # no need for SIDE, just see if both scores are the same
    loss = win == False and tie == False

    def get_val_or(df, field):
        val = df.iloc[0][field]
        if np.isnan(val):
            return 0
        return val
    def get_normalized(x, min, max):
        return (x-min) / (max-min)

    ## DATA FROM PREVIOUS GAME
    if previous_df is not None:
        try:
            wins_before = get_val_or(previous_df, f'{SIDE}_wins_after')
            losses_before = get_val_or(previous_df, f'{SIDE}_losses_after')
            ties_before = get_val_or(previous_df, f'{SIDE}_ties_after')
            record_total_before = get_val_or(previous_df,
                                              f'{SIDE}_record_total_after')
            record_normal_before = get_val_or(previous_df,
                                              f'{SIDE}_record_normal_after')
            schedule_strength_before = get_val_or(previous_df,
                                              f'{SIDE}_schedule_strength_after')

            wins_after = wins_before
            losses_after = losses_before
            ties_after = ties_before
            record_total_after = record_total_before
            record_normal_after = record_normal_before
            schedule_strength_after = schedule_strength_before

        except Exception as e:
            print(e)

    ## CALCULATE NEW DATA (after)
    if win:
        wins_after += 1
    if tie:
        ties_after += 1
    if loss:
        losses_after += 1

    ## CALCULATE RECORD NORMAL
    record_total_after = wins_after - losses_after # eg: +2 or -7 etc.
    record_normal_after = get_normalized(record_total_after, week * -1, week)


    # todo: maybe we should be authoring is_tie as well?

    all_games_df.at[index, [
        f'{SIDE}_wins_before',
        f'{SIDE}_wins_after',
        f'{SIDE}_losses_before',
        f'{SIDE}_losses_after',
        f'{SIDE}_ties_before',
        f'{SIDE}_ties_after',
        f'{SIDE}_record_total_before',
        f'{SIDE}_record_total_after',
        f'{SIDE}_record_normal_before',
        f'{SIDE}_record_normal_after',
        f'{SIDE}_schedule_strength_before',
        f'{SIDE}_schedule_strength_after',
    ]] = [
        wins_before, wins_after, losses_before, losses_after, ties_before,
        ties_after, record_total_before, record_total_after,
        record_normal_before, record_normal_after, schedule_strength_before,
        schedule_strength_after
    ]

    return all_games_df.iloc[index]



## ADD NEW COLUMNS TO ALL GAMES AND CREATE NEW ALL_GAMES_PLUS_DF (agp_df)
new_col_df = pd.DataFrame(columns=[
    'team_wins_before', 'team_losses_before', 'team_ties_before',
    'team_record_total_before', 'team_record_normal_before',
    'team_schedule_strength_before', 'team_wins_after', 'team_losses_after',
    'team_ties_after', 'team_record_total_after', 'team_record_normal_after',
    'team_schedule_strength_after'
])
agp_df = all_games_df.join(new_col_df, how="outer")

def get_team_df(team, year, all_df):
    return all_df[(agp_df['team'] == team) & (agp_df['year'] == year)]

## - - - - - - - - - - - - - - - - - - - -- - - -- - - - - - - -
## C O N F I G
## - - - - - - - - - - - - - - - - - - - -- - - -- - - - - - - -
YEAR = 2018
TEAM = gu.TEAM_NAME['Saints']
team_df = get_team_df(TEAM, YEAR, agp_df)

for index in team_df.index:
    get_df_with_aggregates(index, agp_df)


team_df = get_team_df(TEAM, YEAR, agp_df)
trim_df(team_df)


,year,week,team_win,team,team_wins_before,team_losses_before,team_ties_before,team_wins_after,team_losses_after,team_ties_after,team_record_total_before,team_record_total_after,team_record_normal_before,team_record_normal_after,team_schedule_strength_before,team_schedule_strength_after
4106,2018,1,0,New Orleans Saints,0,0,0,0,1,0,0,-1,0,0.0,0,0
4144,2018,2,1,New Orleans Saints,0,1,0,1,1,0,-1,0,0.0,0.5,0,0
4165,2018,3,1,New Orleans Saints,1,1,0,2,1,0,0,1,0.5,0.666667,0,0
4217,2018,4,1,New Orleans Saints,2,1,0,3,1,0,1,2,0.666667,0.75,0,0
4250,2018,5,1,New Orleans Saints,3,1,0,4,1,0,2,3,0.75,0.8,0,0
4301,2018,7,1,New Orleans Saints,4,1,0,5,1,0,3,4,0.8,0.785714,0,0
4335,2018,8,1,New Orleans Saints,5,1,0,6,1,0,4,5,0.785714,0.8125,0,0
4358,2018,9,1,New Orleans Saints,6,1,0,7,1,0,5,6,0.8125,0.833333,0,0
4367,2018,10,1,New Orleans Saints,7,1,0,8,1,0,6,7,0.833333,0.85,0,0
4412,2018,11,1,New Orleans Saints,8,1,0,9,1,0,7,8,0.85,0.863636,0,0


In [21]:
team_df = get_team_df(TEAM, YEAR, agp_df)
team_df.index

Int64Index([4118, 4154, 4169, 4220, 4231, 4272, 4283, 4323, 4356, 4409, 4440,
            4451, 4499, 4516, 4575, 4590],
           dtype='int64')

In [43]:
agp_df.columns

Index(['date', 'year', 'week', 'team', 'team_score', 'opponent',
       'opponent_score', 'win', 'home', 'overtime', 'year_week', 'team_win',
       'opponent_win', 'temperature', 'wind', 'humidity', 'roof',
       'is_outdoors', 'surface', 'is_grass', 'team_first_downs',
       'team_total_yards', 'team_turnovers_lost', 'team_time_of_possession',
       'team_rush_count', 'team_rush_yards', 'team_rush_td',
       'team_pass_completions', 'team_pass_count', 'team_pass_yards',
       'team_pass_td', 'team_interceptions_lost', 'team_sack_count',
       'team_sack_yards', 'team_fumble_count', 'team_fumble_lost',
       'team_penalty_count', 'team_penalty_yards', 'team_third_down_count',
       'team_third_down_conversions', 'team_third_down_ratio',
       'team_fourth_down_count', 'team_fourth_down_conversions',
       'team_fourth_down_ratio', 'opponent_first_downs',
       'opponent_total_yards', 'opponent_turnovers_lost',
       'opponent_time_of_possession', 'opponent_rush_count',
   